# RED NEURONAL

Esta es una red neuronal simple construida desde cero usando solo NumPy. Se compone de una capa oculta con 128 neuronas y ReLU como función de activación, y una capa de salida con Softmax para clasificación en 10 categorías del dataset Fashion MNIST.

- Importamos las librerias necesarias

In [42]:

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist

- 1 Carga

 Se carga el dataset Fashion MNIST 

In [43]:
# Cargar el dataset Fashion MNIST
(train_X, train_y), (test_X, test_y) = mnist.load_data()

- train_X, train_y: imágenes y etiquetas para el entrenamiento .
- test_X, test_y: imágenes y etiquetas para el test .

- 2 Normalizacion de datos

Se normalizan los valores de píxeles a un rango de 0 a 1.

In [44]:
# Normalizar los datos (0-1) y convertir a 1D
X_train = train_X.reshape(-1, 28 * 28) / 255.0
X_test = test_X.reshape(-1, 28 * 28) / 255.0

- train_X.reshape(-1, 28 * 28): Convierte cada imagen de 28x28 en un vector de 784 elementos .
- / 255.0: Normaliza los valores de píxeles (de 0-255 a 0-1), lo que ayuda al entrenamiento.

- 3 One-Hot Encoding

Las etiquetas se convierten a formato one-hot encoding.

In [45]:
# Convertir etiquetas a codificación one-hot
def one_hot(y, num_classes=10):
    return np.eye(num_classes)[y]

y_train_one_hot = one_hot(train_y)
y_test_one_hot = one_hot(test_y)

- Las redes neuronales trabajan mejor con etiquetas categóricas en formato One-Hot.
- np.eye(num_classes)[y]crea un array de 10 elementos donde solo el índice de la clase es 1, el resto 0.
- Ejemplo:
Si train_y[0] = 3, su versión one-hot es:
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

- 4 Division en entrenamiento y validacion

 

In [46]:
# Dividir en entrenamiento y validación
X_val, y_val = X_train[50000:], y_train_one_hot[50000:]
X_train, y_train = X_train[:50000], y_train_one_hot[:50000]


- Toma los últimos 10,000 ejemplos como conjunto de validación.
- Usa los primeros 50,000 ejemplos para entrenamiento.

- 5 Implementacion de la Red Neuronal

In [47]:
class NeuralNetwork:
    def __init__(self, n_inputs=784, n_outputs=10, hidden_units=128, reg=0.001):
        np.random.seed(0)
        self.reg = reg  # Regularización L2

        # Pesos y sesgos de la capa oculta
        self.w1 = np.random.randn(hidden_units, n_inputs) / np.sqrt(n_inputs)
        self.b1 = np.zeros(hidden_units)

        # Pesos y sesgos de la capa de salida
        self.w2 = np.random.randn(n_outputs, hidden_units) / np.sqrt(hidden_units)
        self.b2 = np.zeros(n_outputs)


- Creamos una clase NeuralNetworkcon:
  - 784 entradas (28x28 píxeles).
  - 128 neuronas ocultas .
  - 10 neuronas de salida (una por cada número del 0 al 9).
- Se inicializan los pesos ( w1, w2) y los sesgos ( b1, b2).
- np.random.randn(...) / np.sqrt(...)es una técnica de inicialización de pesos llamada Xavier Inicialización que ayuda a mejorar el aprendizaje.


- Funciones de activacion

In [48]:
def relu(self, x):
    return np.maximum(0, x)

def softmax(self, x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / exp_x.sum(axis=1, keepdims=True)


- 1. ReLU (Unidad lineal rectificada)
  - Se usa en la capa oculta .
  - max(0, x): Si el valor es negativo, se convierte en 0.
- 2. Máximo suave
  -  Se usa en la capa de salida .
  - Convierte los valores en probabilidades (la suma es 1).

- 6 Propagacion hacia adelante (Paso hacia adelante)

In [49]:
def forward(self, X):
    self.h = self.relu(X @ self.w1.T + self.b1)
    self.output = self.softmax(self.h @ self.w2.T + self.b2)
    return np.argmax(self.output, axis=1)


1. X @ self.w1.T + self.b1: Multiplicamos los datos por los pesos y sumamos los sesgos.
2. self.relu(...):Aplicamos la función ReLU.
3. self.h @ self.w2.T + self.b2: Pasamos la salida de la capa oculta a la capa de salida.
4. self.softmax(...): Convertimos en probabilidades.

- 7  Propagacion hacia atras (Paso hacia atras)

In [50]:
def train(self, X, Y, X_val, Y_val, epochs=50, lr=0.01):
    train_losses, val_losses = [], []

    for epoch in range(epochs):
        # FORWARD PASS
        h = self.relu(X @ self.w1.T + self.b1)
        f = self.softmax(h @ self.w2.T + self.b2)

        # BACKPROPAGATION
        d_out = f - Y
        d_h = (h > 0) * (d_out @ self.w2)

        Dw2 = d_out.T @ h / len(X) + self.reg * self.w2
        Db2 = d_out.mean(axis=0)
        Dw1 = d_h.T @ X / len(X) + self.reg * self.w1
        Db1 = d_h.mean(axis=0)

        # ACTUALIZAR PESOS
        self.w1 -= lr * Dw1
        self.b1 -= lr * Db1
        self.w2 -= lr * Dw2
        self.b2 -= lr * Db2

        # CALCULAR PÉRDIDAS
        loss = -np.mean(np.sum(Y * np.log(f + 1e-8), axis=1))
        val_pred = self.softmax(self.relu(X_val @ self.w1.T + self.b1) @ self.w2.T + self.b2)
        val_loss = -np.mean(np.sum(Y_val * np.log(val_pred + 1e-8), axis=1))

        train_losses.append(loss)
        val_losses.append(val_loss)

        print(f"Época {epoch + 1}, Pérdida: {loss:.4f}, Validación: {val_loss:.4f}")


1. Se calculan los gradientes de los pesos ( Dw1, Dw2) y sesgos ( Db1, Db2).
2. Se actualizan los pesos con lr(tasa de aprendizaje).
3. Se mide la función de pérdida en entrenamiento y validación.
4. Se imprimen los resultados por época.

# Conclusion

Hemos construido una red neuronal desde cero utilizando únicamente NumPy para clasificar imágenes del dataset Fashion MNIST. A continuación, se detalla cómo hemos cumplido con los requisitos obligatorios y las habilidades necesarias:

- Arquitectura de la Red Neuronal:

Diseñamos una red neuronal con una capa oculta de 128 neuronas y una capa de salida de 10 neuronas, adecuada para la clasificación de las 10 categorías del dataset Fashion MNIST.
- Propagación hacia Adelante y Hacia Atrás:

Implementamos la propagación hacia adelante (forward propagation) para calcular las salidas de la red.
Implementamos la retropropagación (backpropagation) para calcular los gradientes y ajustar los pesos y sesgos de la red.
- Funciones de Activación:

Utilizamos la función de activación ReLU en la capa oculta para introducir no linealidad.
Utilizamos la función de activación Softmax en la capa de salida para obtener probabilidades de clasificación.
- Función de Pérdida:

Implementamos la función de pérdida de entropía cruzada para medir el error entre las predicciones y las etiquetas verdaderas.
- Descenso de Gradiente:

Utilizamos el descenso de gradiente para actualizar los pesos y sesgos de la red en cada iteración del entrenamiento.
- Entrenamiento y Evaluación:

Entrenamos la red neuronal durante 50 épocas, observando una disminución constante en la pérdida.
Evaluamos el modelo en el conjunto de prueba, obteniendo una precisión adecuada para una red neuronal simple.